## Module 3 HW

In [12]:
! uv add pandas pyarrow google-cloud-bigquery db-dtypes google-cloud-bigquery-storage
import os, sys, time, requests
import pandas as pd
import db_dtypes
from google.cloud import bigquery, bigquery_storage
from google.api_core.exceptions import NotFound, Forbidden

Resolved 141 packages in 29ms
Audited 128 packages in 45ms


In [7]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "extras/gcp.json"

In [8]:
client = bigquery.Client.from_service_account_json(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

---

##### Tables:

In [25]:
# External Table.
ext = "abiding-bongo-485522-a4.nytaxi.external_yellow_tripdata"

In [26]:
# Materialized Table.
mat = "abiding-bongo-485522-a4.nytaxi.yellow_tripdata_non_partitioned"

---

### Question 1

What is the count of records for the 2024 Yellow Taxi Data? 

In [27]:
df = client.query(f"SELECT COUNT(*) FROM `{mat}`").to_dataframe()
print(df)

        f0_
0  20332093


Answer: `20,332,093`

---

### Question 2

Write a query to count the distinct number of PULocationIDs for the entire dataset on both the tables.

What is the estimated amount of data that will be read when this query is executed on the External Table and the Table?

In [37]:
job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)

In [56]:
# For materialized table.
query_job = client.query(
    f"SELECT COUNT(DISTINCT PULocationID) FROM `{mat}`",
    job_config=job_config
)
print(f"Materialized: {query_job.total_bytes_processed / (1024**2):.2f} MB")


Materialized: 155.12 MB


In [55]:
# For external table.
query_job = client.query(
    f"SELECT COUNT(DISTINCT PULocationID) FROM `{ext}`",
    job_config=job_config
)
print(f"External: {query_job.total_bytes_processed / (1024**2):.2f} MB")


External: 0.00 MB


Answer: `0 MB for the External Table and 155.12 MB for the Materialized Table`

---

### Question 3

Write a query to retrieve the PULocationID from the table (not the external table) in BigQuery. Now write a query to retrieve the PULocationID and DOLocationID on the same table.

Why are the estimated number of Bytes different?

In [59]:
# Query 1.
query_job = client.query(
    f"SELECT PULocationID FROM `{mat}`",
    job_config=job_config
)
print(f"Query 1: {query_job.total_bytes_processed / (1024**2):.2f} MB")


Query 1: 155.12 MB


In [60]:
# Query 2.
query_job = client.query(
    f"SELECT PULocationID, DOLocationID FROM `{mat}`",
    job_config=job_config
)
print(f"Query 2: {query_job.total_bytes_processed / (1024**2):.2f} MB")


Query 2: 310.24 MB


Answer: `BigQuery is a columnar database, and it only scans the specific columns requested in the query. Querying two columns (PULocationID, DOLocationID) requires reading more data than querying one column (PULocationID), leading to a higher estimated number of bytes processed.`

---

### Question 4

How many records have a fare_amount of 0?

In [61]:
df = client.query(f"SELECT COUNT(*) FROM `{mat}` WHERE fare_amount = 0").to_dataframe()
print(df)

    f0_
0  8333


Answer: `8,333`

---

### Question 5

What is the best strategy to make an optimized table in Big Query if your query will always filter based on tpep_dropoff_datetime and order the results by VendorID (Create a new table with this strategy)

In [80]:
opt = "abiding-bongo-485522-a4.nytaxi.yellow_tripdata_optimized"

In [81]:
cluster_query = f"""
CREATE OR REPLACE TABLE `{opt}`
PARTITION BY DATE(tpep_dropoff_datetime)
CLUSTER BY VendorID
AS
SELECT * FROM `{mat}`
"""

In [63]:
client.query(cluster_query).result()
print("Clustered table created.")

Clustered table created.


In [86]:
# Test optimization for full year.
query_job = client.query(
    f"SELECT COUNT(*) FROM `{mat}` WHERE tpep_dropoff_datetime BETWEEN '2024-01-01' AND '2024-01-07' AND VendorID = 1",
    job_config=job_config
)
print(f"Test: {query_job.total_bytes_processed / (1024**2):.2f} MB")

Test: 310.24 MB


In [87]:
# Test optimization for full year.
query_job = client.query(
    f"SELECT COUNT(*) FROM `{opt}` WHERE tpep_dropoff_datetime BETWEEN '2024-01-01' AND '2024-01-07' AND VendorID = 1",
    job_config=job_config
)
print(f"Test: {query_job.total_bytes_processed / (1024**2):.2f} MB")

Test: 9.30 MB


Answer: `Partition by tpep_dropoff_datetime and Cluster on VendorID`

___

### Question 6

Write a query to retrieve the distinct VendorIDs between tpep_dropoff_datetime 2024-03-01 and 2024-03-15 (inclusive). Use the materialized table you created earlier in your from clause and note the estimated bytes. Now change the table in the from clause to the partitioned table you created for question 5 and note the estimated bytes processed. What are these values?

Answer:

---

### Question 7

Where is the data stored in the External Table you created?

Answer:

---

### Question 8

It is best practice in Big Query to always cluster your data:

Answer:

---

### Question 9

Write a `SELECT count(*)` query FROM the materialized table you created. How many bytes does it estimate will be read? Why?

Answer:

---